In [6]:
from elasticsearch import Elasticsearch
import requests

In [7]:
es_client = Elasticsearch('http://localhost:9200')

In [45]:
!curl 'http://localhost:9200'


{
  "name" : "563bd40f68f0",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "64p2SczsTm-HUYZlBWSxeg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


the version.build_hash are :42f05b9372a9a4a470db3b52817899b99a76ee73

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = 'course-homework-1'
es_client.indices.create(index=index_name,body=index_setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-homework-1'})

In [9]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████████████████████| 948/948 [00:29<00:00, 31.88it/s]


Q2 Which function do you use for adding your data to elastic?
in order to adding data to my index on elastic we can use index method

In [5]:
def elastic_search(query):
    search_query = {
        "size":5,
        "query":{
            "bool":{
                "must":{
                    "multi_match":{
                        "query":query,
                        "fields": ["question^4","text"],
                        "type": "best_fields"
                    }
                }
            }
        }
        
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for doc in response['hits']['hits']:
        result_docs.append(doc['_score'])
    return result_docs

In [11]:
query = "How do I execute a command in a running docker container?"
elastic_search(query)

[84.050095, 75.54128, 72.08518, 51.04628, 49.938507]

Q3 What's the score for the top ranking result?
the top ranking result score are 84.05

In [8]:
def elastic_search_with_filter(query):
    search_query = {
        "size":3,
        "query":{
            "bool":{
                "must":{
                    "multi_match":{
                        "query":query,
                        "fields": ["question^4","text"],
                        "type": "best_fields"
                    }
                },
                "filter":{
                    "term":{
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
        
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for doc in response['hits']['hits']:
        result_docs.append(doc['_source'])
    return result_docs

In [10]:
index_name='course-homework-1'
query = "How do I execute a command in a running docker container?"
search_results = elastic_search_with_filter(query)

Q4 
- How do i debug a docker container?
docker ps to see status of the container and
docker logs [containers] to see the logs of the containers
- How do i copy files from a different folder into docker container's working dirrectory?
im quite unsure about this question but if its from local file system to docker container then we can use
docker cp <src-path> <container>:<dest-path>
or copy file from container to local file system
docker cp <container>:<src-path> <local-dest-path>
i've never use these command to copy from one image to another but if its possible then it should be
docker cp <container-src>:<src-path> <container-dest>:<dest-path>
- How do Lambda container images work?
A Lambda container image is a package that includes the Lambda Runtime API, necessary dependencies, an operating system, and function handlers required to run Docker containers in AWS Lambda. These packages allow development teams to deploy and execute arbitrary code and libraries into the Lambda runtime.When creating a Lambda function, AWS supports the use of a container image as the deployment package. With this functionality, you can use the Lambda API or the Lambda console to create a function that is defined by the container image. Once the image is deployed, the underlying code can be further updated and tested to configure various Lambda functions.
- How can i annotate a graph?
adding labels. create tittles that work as headlines, adding detail

In [11]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [12]:
context = "\n\n".join([context_template.format(question=item['question'], text=item['text']) for item in search_results]).strip()

In [13]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [14]:
prompt = prompt_template.format(question = query, context = context)

In [15]:
len(prompt)

1462

Q5 What's the length of the resulting prompt? (use the len function)?
the length of the prompt are 1462

In [4]:
!pip install tiktoken

In [16]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [19]:
len(encoding.encode(prompt))

322

Use the encode function. How many tokens does our prompt have?
322

In [18]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [20]:
from openai import OpenAI

In [22]:
client = OpenAI()

In [27]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role":"user", "content":prompt}]
    )
    return response.choices[0].message.content

In [28]:
llm(prompt)

'To execute a command in a running Docker container, follow these steps:\n\n1. Identify the running container by listing all containers and finding the appropriate `container-id`.\n   ```\n   docker ps\n   ```\n\n2. Once you have the `container-id`, use the `docker exec` command to run the desired command inside the container. If you want to start an interactive bash session, you can use the following command:\n   ```\n   docker exec -it <container-id> bash\n   ```\n\nThis allows you to execute commands within the specified running container.'

In [35]:
def calculate_cost(token_send, token_recieve, total_request):
    total = total_request * (((token_send/1000) * 0.005) + ((token_recieve/1000) *0.015))
    return f"Your total cost with {token_send} input token and {token_recieve} output token for {total_request} requests are : {total}$"

In [36]:
calculate_cost(150, 250, 1000)

'Your total cost with 150 input token and 250 output token for 1000 requests are : 4.5$'